<div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task Definition: Designing Multi-Step Tasks using Brave Search Integration

### Overview



### Task Flow 

1. **Input**: 
   - User provides a topic for the three research questions to be based on

2. **Tool Call**:
   - Preforms a web search for each of the question

3. **Evaluate**:
   - Extract relevant information 

4. **Conditional Logic**:
   - Check to make sure that enough information has been gathered 

5. **Log**:
   - Records a summery

6. **Return**:
   - Returns the final output
         

    

## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/DarkZelda11/julep/blob/dev/cookbooks/06-Designing_Multi-Step_Tasks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:



### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>


Installing the Julep Client

In [4]:
!pip install julep -U --quiet

### Note:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [5]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

## Creating Julep Client with the API Key 

In [6]:
from julep import Client

api_key = " " # Your Api key here

# Create a client
client = Client(api_key=api_key, environment='dev')

## Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent). 

In [ ]:
# Defining the agent 
name = "Jarvis"
about = "The original AI conscious the Iron Man."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}


# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

# Add a web search tool to the client
client.agents.tools.create(
    agent_id=AGENT_UUID,
    name="web_search",
    integration={
        "provider": "brave",
        "method": "search",
        "setup": {"api_key": "your_brave_api_key"},
    }, 
)

## Defining the task 

Tasks in Julep are GitHub Actions style workflows that define long-running, multistep actions.
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [ ]:
import yaml

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [ ]:
# Defining a Task with various step types
task_def = yaml.safe_load("""
name: Multi-Step Task Demonstration

input_schema:
  type: object
  properties:
    topic:
      type: string
      description: The topic to research and summarize.

tools:
  - name: web_search
    type: integration
    integration:
      provider: brave
      setup:
        api_key: "your_api_key"

main:
  # Step 1: Prompt - Initial research question
  - prompt:
      - role: system
        content: "You are a research assistant. Your task is to formulate three specific research questions about the given topic: {{inputs[0].topic}}"
    unwrap: true

  # Step 2: Tool Call - Web search for each question
  - foreach:
      in: _.split('\\n')
      do:
        tool: web_search
        arguments:
          query: _

  # Step 3: Evaluate - Extract relevant information
  - evaluate:
      relevant_info: "[output for output in _]"

  # Step 4: Conditional Logic - Check if enough information is gathered
  - if: "len(_.relevant_info) >= 3"
    then:
      prompt:
        - role: system
          content: "Summarize the following information about {{inputs[0].topic}}:\n{{_.relevant_info}}"
      unwrap: true
    else:
      prompt:
        - role: system
          content: "Not enough information gathered. Please provide a brief overview of {{inputs[0].topic}} based on your knowledge."
      unwrap: true

  # Step 5: Log - Record the summary
  - log: "Summary for {{inputs[0].topic}}: {{_}}"

  # Step 6: Return - Final output
  - return: 
      summary: "_"
      topic: "inputs[0].topic"
""")

Creating/Updating a task.

In [ ]:
# Creating the task update
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates an execution workflow for the Task defined in the yaml file.

In [ ]:
# Creating an Execution object
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "topic": "Artificial Intelligence in Healthcare"
    }
)

In [ ]:
# Getting the execution details
execution = client.executions.get(execution.id)
# printing the output
print("Execution Output:")
print(client.executions.transitions.list(execution_id=execution.id).items[0].output)

Retrieves and lists all the steps of a defined task that have been executed up to that point in time. Unlike streaming, this function does not continuously monitor the execution; it only provides a snapshot of the steps completed so far without displaying real-time updates as the task progresses.

In [ ]:
client.executions.transitions.list(execution_id=execution.id).items

Continuously monitor and stream the steps of a defined task. It retrieves and displays the transitions or execution steps of the task identified by execution. id in real-time, showing each step sequentially until the task is either completed or an error causes it to terminate.

In [ ]:
client.executions.transitions.stream(execution_id=execution.id)